
# Set up MERIT Hydro catchments

This notebook is used to set up the MERIT Hydro catchments as `.parquet` and `.geojson` files on the google cloud.  
We transform the CRS to EPSG:3034 to be easily compatible with HYRAS, add an OID and save the catchment for each station as .parquet and .geojson on gcloud. 

In [1]:
from app.db import HyrasDB

import tempfile
from pathlib import Path
from tqdm import tqdm
from geocube.api.core import make_geocube

Get catchments from gcloud (`camels_output_data/features/merit_hydro_catchments.gpkg`).

In [2]:
db = HyrasDB()

catchments = db.catchments


Transform CRS to `EPSG:3034` to be easily compatible with HYRAS data and add an OID.

In [3]:
catchments.to_crs('epsg:3034', inplace=True)
catchments['oid'] = range(len(catchments))


Save catchment geometry for each station as `.parquet` and `.geojson` on gcloud.

In [10]:
# set path to store catchments on gcloud
base_path = Path('test/db/')

# loop over all catchments and save as parquet and geojson
for i, row in tqdm(catchments.iterrows()):
    # make geocube for the catchment
    ds = make_geocube(catchments.where(catchments.oid == row.oid).dropna(), measurements=['oid'], resolution=(-500, 500))

    # save to parquet
    with tempfile.NamedTemporaryFile(suffix='.parquet') as f:
        # transform to parquet
        ds.to_dataframe().dropna().reset_index().astype(int)[['x', 'y', 'oid']].to_parquet(f)
        f.seek(0)

        # generate path (camels_id/camels_id.parquet)
        path = base_path / f"{row.id}/{row.id}.parquet"
        
        # upload to gcloud
        db.proc.upload(str(path), f.name)
    
    # save to geojson
    with tempfile.NamedTemporaryFile(suffix='.geojson') as f:
        # transform to geojson
        catchments.where(catchments.id == row.id).dropna().to_file(f)

        # generate path (camels_id/camels_id.geojson)
        path = base_path / f"{row.id}/{row.id}.geojson"

        # upload to gcloud
        db.proc.upload(str(path), f.name)

    # TODO: remove break and run
    break

1it [00:00,  4.51it/s]

0


2it [00:00,  4.46it/s]

1


2it [00:00,  2.82it/s]

2
